In [282]:
import numpy as np
import scipy.sparse

In [283]:
train_X, test_X = scipy.sparse.load_npz('training_feats.npz').toarray(), scipy.sparse.load_npz('test_feats.npz').toarray() # DO NOT modify the PATH
train_y, test_y = np.load('training_labels.npy', allow_pickle=True), np.load('test_labels.npy', allow_pickle=True)
print(train_X.shape) # 11314 条post，每个post由1000个weights表示（1000个特征）
print(train_y.shape) # 11314 条lable 一共有20 类
print(test_X.shape)  # 7532  
print(test_y.shape)  # 7532  

(11314, 1000)
(11314,)
(7532, 1000)
(7532,)


In [284]:
# resturn a list with length 20
# example: 
# classified_data = getSplitedClass()
# classified_data[19] contains all posts whose label is 19

# classified_data[i] is numpyArray with shape (number of posts in that class, 1000)
def getSplitedClass():
    res = []
    for label in range(0, 20):
        post_index = np.where(train_y == label)[0].tolist()
        a_class = train_X[post_index[0]]
        for index in range(1, len(post_index)):
            a_class = np.vstack((a_class,train_X[post_index[index]]))
        res.append(a_class)
    return res

In [285]:
for post in range(len(train_X)):
    train_X[post] = np.where(train_X[post]!=0.0,1,train_X[post])
for post in range(len(test_X)):
    test_X[post] = np.where(test_X[post]!=0.0,1,test_X[post])
classified_data = getSplitedClass()
classified_data

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [1., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 arr

In [286]:
def getPriorProb():
    prior_prob = []
    for label in range(0, 20):
        prior_prob.append(len(classified_data[label]) / len(train_X))
    return np.array(prior_prob)

In [287]:
prior_prob = getPriorProb()
prior_prob

array([0.04242531, 0.05161747, 0.05223617, 0.05214778, 0.05108715,
       0.05241294, 0.05170585, 0.05250133, 0.05285487, 0.05276648,
       0.05303164, 0.05258971, 0.05223617, 0.05250133, 0.05241294,
       0.05294326, 0.04825879, 0.04984974, 0.04109952, 0.03332155])

In [288]:
def getBernoulli(data, post_number, label_number, feature_number): 
    data_T = classified_data[label_number].T
    param=(np.count_nonzero(data_T[feature_number])+ 1) / (len(data_T[0])+ 20)
    feature_value=data[post_number][feature_number]
    bernoulli = param * feature_value + (1 - param) * (1 - feature_value)
    return bernoulli

In [289]:
def prodb(data, post_number, label_number):
    product=1
    for i in range(0,1000):
        product *= getBernoulli(data, post_number, label_number, i)
    return product

In [290]:
def predict(data,post_number):
    res=0
    best = 0
    for i in range(0,20):
        score = prodb(data, post_number, i) * prior_prob[i]
        if score > best:
            best = score
            res = i
    return res

In [291]:
estimate_class=np.empty(len(test_X))
for i in range(0,len(test_X)):
    if i%100 == 0:
        print(i)
    estimate_class[i] = predict(test_X,i)
estimate_class

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500


array([ 4.,  8.,  1., ...,  9.,  3., 15.])

In [292]:
cnt = 0
for i in range(len(estimate_class)):
    if test_y[i]==estimate_class[i]:
        cnt += 1
acc = float(cnt / len(estimate_class))
acc

0.5011949017525226